In [1]:
import argparse
import os
import random
import time
from distutils.util import strtobool

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stable_baselines3.common.atari_wrappers import (
    ClipRewardEnv,
    EpisodicLifeEnv,
    FireResetEnv,
    MaxAndSkipEnv,
    NoopResetEnv,
)
from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter
from collections import defaultdict


def parse_args():
    args = {
        "exp-name": "AtariDQN",
        "seed": 1,
        "torch-deterministic": True,
        "cuda": True,
        "track": False,
        "wandb-project-name": "cleanRL",
        "wandb-entity": None,
        "capture-video": False,
        "save-model": False,
        "upload-model": False,
        "hf-entity": "",
        "env-id": "BreakoutNoFrameskip-v4",
        "total-timesteps": 10000000,
        "learning-rate": 1e-4,
        "num-envs": 1,
        "buffer-size": 1000000,
        "gamma": 0.99,
        "tau": 1.,
        "target-network-frequency": 1000,
        "batch-size": 32,
        "start-e": 1,
        "end-e": 0.01,
        "exploration-fraction": 0.20,
        "learning-starts": 80000,
        "train-frequency": 4
    }

    assert args["num-envs"] == 1, "vectorized envs are not supported at the moment"

    # Convert the dictionary to a defaultdict with default value as None
    args_dict = defaultdict(lambda: None, args)
    
    
    args_dict = dict(args_dict)

    class Args:
        def __init__(self, **kwargs):
            self.__dict__.update(kwargs)
            for k in list(self.__dict__.keys()):
                self.__dict__[k.replace("-","_")]=self.__dict__[k]
    args = Args(**args_dict)

    return args


def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        if capture_video and idx == 0:
            env = gym.make(env_id, render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        else:
            env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = ClipRewardEnv(env)
        env = gym.wrappers.ResizeObservation(env, (84, 84))
        env = gym.wrappers.GrayScaleObservation(env)
        env = gym.wrappers.FrameStack(env, 4)
        env.action_space.seed(seed)

        return env

    return thunk


# ALGO LOGIC: initialize agent here:
class QNetwork(nn.Module):
    def __init__(self, env):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(4, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, env.single_action_space.n),
        )

    def forward(self, x):
        return self.network(x / 255.0)


def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

In [2]:
args = parse_args()
args.env_id

'BreakoutNoFrameskip-v4'

In [3]:
import stable_baselines3 as sb3

if sb3.__version__ < "2.0":
    raise ValueError(
        """Ongoing migration: run the following command to install the new dependencies:

poetry run pip install "stable_baselines3==2.0.0a1" "gymnasium[atari,accept-rom-license]==0.28.1"  "ale-py==0.8.1" 
"""
    )
    
args = parse_args()
run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
if args.track:
    import wandb

    wandb.init(
        project=args.wandb_project_name,
        entity=args.wandb_entity,
        sync_tensorboard=True,
        config=vars(args),
        name=run_name,
        monitor_gym=True,
        save_code=True,
    )
writer = SummaryWriter(f"runs/{run_name}")
writer.add_text(
    "hyperparameters",
    "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
)

# TRY NOT TO MODIFY: seeding
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

# env setup
envs = gym.vector.SyncVectorEnv(
    [make_env(args.env_id, args.seed + i, i, args.capture_video, run_name) for i in range(args.num_envs)]
)
assert isinstance(envs.single_action_space, gym.spaces.Discrete), "only discrete action space is supported"

q_network = QNetwork(envs).to(device)
optimizer = optim.Adam(q_network.parameters(), lr=args.learning_rate)
target_network = QNetwork(envs).to(device)
target_network.load_state_dict(q_network.state_dict())

rb = ReplayBuffer(
    args.buffer_size,
    envs.single_observation_space,
    envs.single_action_space,
    device,
    optimize_memory_usage=True,
    handle_timeout_termination=False,
)
start_time = time.time()

# TRY NOT TO MODIFY: start the game
obs, _ = envs.reset(seed=args.seed)
for global_step in range(args.total_timesteps):
    # ALGO LOGIC: put action logic here
    epsilon = linear_schedule(args.start_e, args.end_e, args.exploration_fraction * args.total_timesteps, global_step)
    if random.random() < epsilon:
        actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
    else:
        q_values = q_network(torch.Tensor(obs).to(device))
        actions = torch.argmax(q_values, dim=1).cpu().numpy()

    # TRY NOT TO MODIFY: execute the game and log data.
    next_obs, rewards, terminated, truncated, infos = envs.step(actions)

    # TRY NOT TO MODIFY: record rewards for plotting purposes
    if "final_info" in infos:
        for info in infos["final_info"]:
            # Skip the envs that are not done
            if "episode" not in info:
                continue
            print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
            writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
            writer.add_scalar("charts/episodic_length", info["episode"]["l"], global_step)
            writer.add_scalar("charts/epsilon", epsilon, global_step)

    # TRY NOT TO MODIFY: save data to reply buffer; handle `final_observation`
    real_next_obs = next_obs.copy()
    for idx, d in enumerate(truncated):
        if d:
            real_next_obs[idx] = infos["final_observation"][idx]
    rb.add(obs, real_next_obs, actions, rewards, terminated, infos)

    # TRY NOT TO MODIFY: CRUCIAL step easy to overlook
    obs = next_obs

    # ALGO LOGIC: training.
    if global_step > args.learning_starts:
        if global_step % args.train_frequency == 0:
            data = rb.sample(args.batch_size)
            with torch.no_grad():
                target_max, _ = target_network(data.next_observations).max(dim=1)
                td_target = data.rewards.flatten() + args.gamma * target_max * (1 - data.dones.flatten())
            old_val = q_network(data.observations).gather(1, data.actions).squeeze()
            loss = F.mse_loss(td_target, old_val)

            if global_step % 100 == 0:
                writer.add_scalar("losses/td_loss", loss, global_step)
                writer.add_scalar("losses/q_values", old_val.mean().item(), global_step)
                print("SPS:", int(global_step / (time.time() - start_time)))
                writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

            # optimize the model
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # update target network
        if global_step % args.target_network_frequency == 0:
            for target_network_param, q_network_param in zip(target_network.parameters(), q_network.parameters()):
                target_network_param.data.copy_(
                    args.tau * q_network_param.data + (1.0 - args.tau) * target_network_param.data
                )

if args.save_model:
    model_path = f"runs/{run_name}/{args.exp_name}.cleanrl_model"
    torch.save(q_network.state_dict(), model_path)
    print(f"model saved to {model_path}")
    from cleanrl_utils.evals.dqn_eval import evaluate

    episodic_returns = evaluate(
        model_path,
        make_env,
        args.env_id,
        eval_episodes=10,
        run_name=f"{run_name}-eval",
        Model=QNetwork,
        device=device,
        epsilon=0.05,
    )
    for idx, episodic_return in enumerate(episodic_returns):
        writer.add_scalar("eval/episodic_return", episodic_return, idx)

    if args.upload_model:
        from cleanrl_utils.huggingface import push_to_hub

        repo_name = f"{args.env_id}-{args.exp_name}-seed{args.seed}"
        repo_id = f"{args.hf_entity}/{repo_name}" if args.hf_entity else repo_name
        push_to_hub(args, episodic_returns, repo_id, "DQN", f"runs/{run_name}", f"videos/{run_name}-eval")

envs.close()
writer.close()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
/Users/sni/Documents/Projects/Telluride/BrainDishSiMulator/venv/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 28.24GB > 21.12GB
  warnings.warn(


global_step=139, episodic_return=[1.]
global_step=324, episodic_return=[2.]
global_step=435, episodic_return=[0.]
global_step=673, episodic_return=[3.]
global_step=935, episodic_return=[3.]
global_step=1141, episodic_return=[2.]
global_step=1352, episodic_return=[2.]
global_step=1495, episodic_return=[1.]
global_step=1608, episodic_return=[0.]
global_step=1767, episodic_return=[1.]
global_step=1882, episodic_return=[0.]
global_step=1995, episodic_return=[0.]
global_step=2201, episodic_return=[2.]
global_step=2312, episodic_return=[0.]
global_step=2471, episodic_return=[1.]
global_step=2678, episodic_return=[2.]
global_step=2819, episodic_return=[1.]
global_step=2978, episodic_return=[1.]
global_step=3091, episodic_return=[0.]
global_step=3279, episodic_return=[2.]
global_step=3468, episodic_return=[2.]
global_step=3677, episodic_return=[2.]
global_step=3861, episodic_return=[2.]
global_step=3974, episodic_return=[0.]
global_step=4133, episodic_return=[1.]
global_step=4294, episodic_ret

global_step=34813, episodic_return=[1.]
global_step=34928, episodic_return=[0.]
global_step=35043, episodic_return=[0.]
global_step=35229, episodic_return=[2.]
global_step=35368, episodic_return=[1.]
global_step=35479, episodic_return=[0.]
global_step=35622, episodic_return=[1.]
global_step=35733, episodic_return=[0.]
global_step=35876, episodic_return=[1.]
global_step=35991, episodic_return=[0.]
global_step=36150, episodic_return=[1.]
global_step=36265, episodic_return=[0.]
global_step=36378, episodic_return=[0.]
global_step=36519, episodic_return=[1.]
global_step=36632, episodic_return=[0.]
global_step=36747, episodic_return=[0.]
global_step=36890, episodic_return=[1.]
global_step=37003, episodic_return=[0.]
global_step=37142, episodic_return=[1.]
global_step=37253, episodic_return=[0.]
global_step=37583, episodic_return=[5.]
global_step=37891, episodic_return=[4.]
global_step=38032, episodic_return=[1.]
global_step=38145, episodic_return=[0.]
global_step=38391, episodic_return=[4.]


global_step=68707, episodic_return=[0.]
global_step=68913, episodic_return=[2.]
global_step=69101, episodic_return=[2.]
global_step=69214, episodic_return=[0.]
global_step=69471, episodic_return=[3.]
global_step=69681, episodic_return=[2.]
global_step=69792, episodic_return=[0.]
global_step=69952, episodic_return=[1.]
global_step=70065, episodic_return=[0.]
global_step=70180, episodic_return=[0.]
global_step=70499, episodic_return=[5.]
global_step=70661, episodic_return=[1.]
global_step=70849, episodic_return=[2.]
global_step=70960, episodic_return=[0.]
global_step=71073, episodic_return=[0.]
global_step=71230, episodic_return=[1.]
global_step=71343, episodic_return=[0.]
global_step=71676, episodic_return=[5.]
global_step=71883, episodic_return=[2.]
global_step=71996, episodic_return=[0.]
global_step=72155, episodic_return=[1.]
global_step=72268, episodic_return=[0.]
global_step=72381, episodic_return=[0.]
global_step=72767, episodic_return=[9.]
global_step=72878, episodic_return=[0.]


SPS: 8
SPS: 8
global_step=97702, episodic_return=[2.]
SPS: 8
SPS: 8
SPS: 8
global_step=98036, episodic_return=[5.]
SPS: 8
global_step=98197, episodic_return=[1.]
SPS: 8
SPS: 8
SPS: 8
global_step=98453, episodic_return=[3.]
SPS: 8
SPS: 8
global_step=98612, episodic_return=[1.]
SPS: 8
global_step=98727, episodic_return=[0.]
SPS: 8
global_step=98842, episodic_return=[0.]
SPS: 8
global_step=98957, episodic_return=[0.]
SPS: 8
global_step=99068, episodic_return=[0.]
SPS: 8
global_step=99183, episodic_return=[0.]
SPS: 8
SPS: 8
global_step=99400, episodic_return=[3.]
SPS: 8
SPS: 8
global_step=99541, episodic_return=[1.]
SPS: 8
SPS: 8
global_step=99732, episodic_return=[2.]
SPS: 8
SPS: 8
global_step=99968, episodic_return=[3.]
SPS: 8
SPS: 8
global_step=100156, episodic_return=[2.]
SPS: 8
global_step=100269, episodic_return=[0.]
SPS: 8
SPS: 8
global_step=100426, episodic_return=[1.]
SPS: 8
global_step=100541, episodic_return=[0.]
SPS: 8
SPS: 8
SPS: 8
global_step=100826, episodic_return=[4.]
SPS: